$$
\begin{array}{c}
\LARGE \textbf{Trabajo\ Final\ de\ Comunicaciones\ Digitales}
\end{array}
$$

$$
\begin{aligned}
&\textbf{Introducción}
\end{aligned}
$$

$$
\begin{aligned}
&\textbf{Marco teórico}
\end{aligned}
$$

$$
\begin{aligned}
&\textbf{} \\
&\text{El auge del Internet de las Cosas (IoT) ha impulsado el desarrollo de tecnologías de comunicación de} \\
&\text{bajo consumo y largo alcance, conocidas como Low Power Wide Area Networks (LPWAN). Dentro de este} \\
&\text{marco, LoRa (Long Range) es el estándar que se ha consolidado como una tecnología ideal para la} \\
&\text{comunicación inalámbrica de largo alcance y bajo consumo energético.} \\[1em]
&\text{En este sentido, el artículo propuesto por la cátedra introduce una formulación matemática para la} \\
&\text{modulación LoRa, la cual puede describirse como una modulación chirp con desplazamiento de frecuencia} \\
&\text{(Frequency Shift Chirp Modulation, FSCM).Esta técnica se basa en la codificación de bits en símbolos} \\
&\text{ enteros y en la generación de señales chirp cuya frecuencia depende de dichos símbolos.} \\
&\text{Además, en esta formulación, la información está codificada en el desplazamiento inicial de frecuencia } \\
&\text{del chirp,y no depende de la pendiente ni en la duración de la señal.Este enfoque permite definir una base } \\
&\text{ortogonal de señales discretas, lo que favorece una demodulación eficiente mediante la Transformada Rápida de}\\
&\text{Fourier (FFT).}\\
\end{aligned}
$$

$$
\begin{aligned}
&\textbf{Desarrollo}
\end{aligned}
$$

$$
 \begin{aligned}
 &\text{Este proceso de codificación, tal como se describe matemáticamente en el artículo, puede descomponerse en varias} \\
 &\text{etapas fundamentales. La primera de ellas es el mapeo de bits a un símbolo decimal, y comienza con un vector} \\
 &\text{ $w(nT_s)$ compuesto por una cantidad fija de bits determinada por el parámetro Spreading Factor (SF).} \\
 &\text{Este factor representa la cantidad de  bits por símbolo y se define como: } \\[1em]
 
 &\hspace{8cm} SF = \log_2(M) \\[1em]
 &\text{Donde $M$ es la cantidad total de símbolos posibles (cardinalidad de la modulación). Por ejemplo,} \\
 &\text{si $SF = 5$, existen $2^5 = 32$ símbolos distintos.} \\
 &\text{El valor decimal resultante de este mapeo, denotado como $s(nT_s)$, se obtiene al sumar los valores ponderados} \\
 &\text{de los bits del vector $w(nT_s)$. Este número representa directamente la frecuencia inicial desde la cual se} \\
 &\text{genera la señal chirp para su transmisión.} \\[1em]
 &\textbf{Fórmula de codificación:} \\[1em]
 &\hspace{8cm} s(nT_s) = \sum_{h=0}^{SF-1} w_h(nT_s) \cdot 2^h \\[1em]
 &\text{donde:} \\
 &\hspace{1cm} ~ s(nT_s) \in \{0, ..., 2^{SF} - 1\} \text{ es el símbolo codificado} \\
 &\hspace{1cm} ~ w_h(nT_s) \in \{0,1\} \text{ son los bits del bloque actual, ordenados desde el LSB} \\
 &\hspace{1cm} ~ SF \text{ es el Spreading Factor (cantidad de bits por símbolo)} \\
 &\hspace{1cm} ~ h \text{ es el índice del bit dentro del bloque} \\[1em]
 &\textbf{Parámetros de transmisión:} \\[1em]
 &\text{Supongamos que el ancho de banda del canal utilizado es $B$, lo que implica que transmitimos una muestra cada:} \\[1em]
 &\hspace{10cm} T = \frac{1}{B} \\[1em]
 &\text{Un símbolo $s(nT_s)$ se transmite cada:} \\[1em]
 &\hspace{10cm} T_s = 2^{SF} \cdot T \\[1em]
 &\text{Cada símbolo representa un número entero entre $0$ y $2^{SF} - 1$. Esto no solo determina la frecuencia} \\
 &\text{con la que inicia el chirp, sino también cuán "extendida" estará la señal en el tiempo. A mayor $SF$,} \\
 &\text{mayor será la duración del símbolo y mayor la resistencia al ruido, a costa de menor tasa de transmisión.} \\[1em]
 &\text{Por otro lado, el decodificador es el componente encargado de inferir el mensaje original a partir de la señal   recibida.} \\
 &\text{En un canal aditivo de ruido blanco gaussiano (AWGN), la estrategia óptima es la de Máxima Verosimilitud (ML).} \\
 &\text{Esta consiste en proyectar la señal recibida sobre todas las señales base posibles y elegir aquella cuya proyección} \\
 &\text{maximice la expresión:} \\[1em]
 &\hspace{10cm} \max_{c} \left| \langle y, c \rangle \right| \\[1em]
 &\text{Donde $c$ es una señal candidata y $y$ es la señal observada. Gracias a la ortogonalidad de las señales en FSCM,} \\
 &\text{esta proyección puede implementarse eficientemente con una FFT.}
 \end{aligned}
 $$

$$
\begin{aligned}
&\textbf{Implementación en Python}
\end{aligned}
$$

$$
\begin{aligned}
&\text{La función } \texttt{bits\_to\_symbols}: \text{ Agrupa un arreglo plano de bits en bloques de longitud igual al} \\
&\text{Spreading Factor (SF) y convierte cada bloque en un número decimal. Este paso simula lo que en el paper} \\
&\text{se describe como la generación del símbolo } s(nT_s) \text{ a partir del vector de bits } w(nT_s).
\end{aligned}
$$

In [ ]:
import numpy as np

def bits_to_symbols(bit_array, SF):
    """
    Agrupa bits en bloques de tamaño SF y convierte cada grupo en un símbolo decimal.
    """
    if len(bit_array) % SF != 0:
        raise ValueError(f"Número de bits ({len(bit_array)}) no divisible por el SF ({SF})")

    # Convertir a array de NumPy y reorganizar en filas de SF bits
    bloques = np.array(bit_array).reshape(-1, SF)

    # Calculamos los pesos binarios desde el bit más significativo al menos
    pesos = [2**i for i in range(SF - 1, -1, -1)]

    # Multiplicamos cada grupo por los pesos y sumamos para obtener los símbolos
    symbols = []
    for grupo in bloques:
        decimal = sum(b * p for b, p in zip(grupo, pesos))
        symbols.append(decimal)

    return symbols



In [ ]:
def symbols_to_bits(symbols, SF):
    """Convierte los símbolos de vuelta a su representación binaria."""
      #Para cada símbolo, aplica una máscara binaria para extraer los bits (de mayor a menor).
    bits = ((symbols[:, None] & (1 << np.arange(SF)[::-1])) > 0).astype(int)
    #Devuelve los bits reordenados en una sola dimensión.
    return bits.reshape(-1)

In [10]:
def simulate_encoder_decoder(SF, total_bits):
    """Genera bits aleatorios, codifica, decodifica y calcula el BER."""
    assert total_bits % SF == 0, "El número total de bits debe ser múltiplo de SF"

    # Generar bits aleatorios con distribución uniforme
    tx_bits = np.random.randint(0, 2, total_bits)

    # Codificación
    tx_symbols = bits_to_symbols(tx_bits, SF)

    # Transmisión simulada perfecta (sin ruido)
    rx_symbols = tx_symbols.copy()  # En canal real, se podría agregar ruido o errores

    # Decodificación
    rx_bits = symbols_to_bits(rx_symbols, SF)

    # Cálculo de BER
    bit_errors = np.sum(tx_bits != rx_bits)
    ber = bit_errors / total_bits

    # Resultados
    print("Bits transmitidos (primeros 64):  ", tx_bits[:64])
    print("Bits decodificados (primeros 64):", rx_bits[:64])
    print(f"\nTotal de bits transmitidos: {total_bits}")
    print(f"Errores totales: {bit_errors}")
    print(f"BER (Bit Error Rate): {ber:.6f}")

# Parámetros de simulación
SF = 7  # Spreading Factor
num_bits = SF * 1000  # Enviar 1000 símbolos

simulate_encoder_decoder(SF, num_bits)

Bits transmitidos (primeros 64):   [0 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1]
Bits decodificados (primeros 64): [0 1 0 0 1 0 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1]

Total de bits transmitidos: 7000
Errores totales: 0
BER (Bit Error Rate): 0.000000


$$
\begin{aligned}
&\text{La segunda etapa consiste en generar la forma de onda a transmitir a partir del numero decimal entero } s(nT_s), \\
&\text{esta onda se denomina chirp, donde el termino S(nTs) indica la frecuencia inicial a partir de la cual dicha señal} \\
&\text{comienza a transmitir los símbolos codificados.}
\end{aligned}
$$

$$
\begin{aligned}
&\textbf{Forma de onda modulada por FSCM:} \\[1em]
&\hspace{5cm} c(nT_s + kT) = \frac{1}{\sqrt{2^{SF}}} \cdot e^{j2\pi[(s(nT_s)+k) \bmod 2^{SF}]\frac{kT}{B}} \quad (2) \\[1em]
&\text{Si reformulamos la ecuación 2,  usando la relación: } T = \frac{1}{B} \\
&\text{\hspace{0.5cm} Sabemos que: } T = \frac{1}{B} \Longrightarrow \quad T \cdot B = 1  \\
&\text{\hspace{0.5cm} Entonces: } kT \cdot B = k \\
&\text{\hspace{0.5cm} Sustituyendo en la ecuación } (2): \text{damos origen a la ecuación } (3)  \\
&\hspace{5cm} c(nT_s + kT) = \frac{1}{\sqrt{2^{SF}}} \cdot e^{j2\pi[(s(nT_s)+k) \bmod 2^{SF}] \cdot \frac{k}{2^{SF}}} (3) \\[1em]


&\text{donde:} \\
&\hspace{1cm} c(nT_s + kT) \text{ es la señal chirp modulada en el tiempo } nT_s + kT \\
&\hspace{1cm} \frac{1}{\sqrt{2^{SF}}} \text{ es el factor de normalización de energía (asegura energía unitaria para cada símbolo)} \\
&\hspace{1cm} s(nT_s) \text{ es el símbolo codificado (de la Ecuación 1)} \\
&\hspace{1cm} k \text{ es el índice de muestra dentro del símbolo } (k = 0, 1, ..., 2^{SF}-1) \\
&\hspace{1cm} T \text{ es el período de muestreo} \\
&\hspace{1cm} B \text{ es el ancho de banda del canal} \\
&\hspace{1cm} \bmod 2^{SF} \text{ indica la operación módulo } 2^{SF} \\[2em]

&\text{La ecuación } (2)\quad y\quad (3) \text{ del paper define la forma de onda transmitida por el modulador FSCM, donde cada} \\
&\text{ símbolo} s(nT_s) \in {0,1,...,2^SF} \text{codifica SF bits binarios} \\

&\textbf{Características principales:} \\[1em]
&\text{1. Chirp complejo: La señal tiene frecuencia linealmente creciente dentro de cada símbolo} \\
&\text{2. Frecuencia instantánea: Depende de } (s(nT_s)+k) \bmod 2^{SF}, \text{ que representa un corrimiento de frecuencia} \\
&\text{\hspace{0.5cm} específico para cada símbolo y hace que la frecuencia aumente de manera lineal en el tiempo } \\
&\text{3. Ortogonalidad: Los diferentes símbolos generan chirps ortogonales entre sí, facilitando la decodificación} \\
&\text{4. Normalización: El factor } \frac{1}{\sqrt{2^{SF}}} \text{ mantiene la energía constante por símbolo} \\ 
&\text{\hspace{0.5cm}es el proceso mediante el cual una señal se escala para que su energía total sea igual a 1 } \\
&\text{\hspace{0.5cm} Esto permite: comparar distintas señales de forma justa, controlar la energía o potencia  } \\
&\text{\hspace{0.5cm} promedio transmitida y aplicar citerios óptimos  } (como\quad detección\quad ML) \text{ sin sesgos por amplitud} \\
&\text{a. Esto asegura que todos los símbolos tengan energía unitaria, lo que es necesario para el análisis óptimo en  }  \\
&\text{\hspace{0.5cm} presencia de ruido blanco  } (AWGN) \\
&\textbf{Ventajas:} \\[1em]
&\text{1. Robustez en canales selectivos: propiedad de una modulación que permite mantener baja tasa de error en } \\
&\text{\hspace{0.5cm} presencia de canales con respuesta en frecuencia no plana, es decir, donde ciertas subbandas del } \\
&\text{\hspace{0.5cm} espectro sufren atenuación significativa. } \\
&\text{2. Demodulación eficiente: implementación del proceso de detección óptima con complejidad computacional reducida, } \\
&\text{\hspace{0.5cm} frecuentemente lograda mediante rápidas (FFT), evitando correlaciones directas sobre una base completa de señales } \\

&\textbf{Resumen:} \\[1em]
&\text{Modela cada símbolo como un chirp complejo que varía su frecuencia en el tiempo con una pendiente fija, pero que} \\
&\text{\hspace{0.5cm} comienza en una frecuencia inicial diferente para cada símbolo. Esto hace que los símbolos sean  } \\
&\text{\hspace{0.5cm} ortogonales, tengan la misma energía y puedan demodularse de forma eficiente usando una FFT} \\
\end{aligned}
$$

$$
\begin{aligned}
&\text{A partir de la ecuación (3), se plantea la ecuación que describe la señal recibida por el receptor, luego de que la señal modulada} \\
& c(nT_s + kT)\text{ se ha transmito a través de un AWGN(Additive White Gaussian Noise ) }  \\
&\text{donde se parte de la suposición de que las señales son de igual energía y con una sincronización perfecta} \\
&\text{en tiempo y frecuencia; así como también una fuente que emite símbolos con la misma probabilidad.} \\[1em]
&\hspace{5cm} r(nT_s + kT) = c(nT_s + kT) + w(nT_s + kT) \\[1em]
&\text{donde:} \\
&\hspace{1cm} c(nT_s + kT): \text{ la chirp transmitida para el símbolo } s(nT_s) \\
&\hspace{1cm} w(nT_s + kT): \text{ ruido blanco gaussiano} \\
&\hspace{1cm} r: \text{ señal recibida}
\end{aligned}
$$

$$
\begin{aligned}
&\text{Luego de definir la forma de onda transmitida } c(nT_s + kT) \text{asociada a un símbolo } s(nT_s), \\
&\text{ como una secuencia chirp ortogonalizada y normalizada (Ecuación 3). Se analiza la ortogonalidad entre las} \\
&\text{las diferentes formas de onda posible para distintos símbolos:} \\
&\text{\hspace{0.5cm} Es decir, si } s_1 \neq  s_2, \text{ entonces los chirp asociados deben ser ortogonales}\\
&\text{\hspace{0.5cm} Esto se demuestra en las ecuaciones (4) a (9) del paper [1], culminando en:} \\
&\hspace{5cm} \langle c_i, c_j \rangle = 0 \quad \text{para } i \neq j\\[1em]
&\text{\hspace{0.5cm} propiedad clave para poder realizar detección óptima en presencia de ruido. } \\

\end{aligned}
$$


\begin{aligned}
&\text{En consecuencia, se plantea la ecuación que describe la señal recibida por el receptor óptimo para la} \\
&\text{modulación utilizada, proceso que culmina en la detección óptima en presencia de ruido blanco (AWGN),} \\
&\text{y en la implementación eficiente mediante el \emph{n-tuple former}.} \\
&\text{Este planteo parte de la suposición de que las señales poseen igual energía, con sincronización perfecta} \\
&\text{en tiempo y frecuencia, así como también una fuente que emite símbolos con igual probabilidad.} \\[1em]
&\hspace{5cm} r(nT_s + kT) = c(nT_s + kT) + w(nT_s + kT)  (10)\\[1em]
&\text{donde:} \\
&\hspace{1cm} c(nT_s + kT): \text{ chirp transmitida para el símbolo } s(nT_s), \\
&\hspace{1cm} w(nT_s + kT): \text{ ruido blanco gaussiano complejo de media cero y varianza constante}, \\
&\hspace{1cm} r(nT_s + kT): \text{ señal recibida en el instante discreto } k.
\end{aligned}



$$
\begin{aligned}
&\textbf{Procedimiento de detección óptima:} \\[1em]
&\text{Donde el desarrollo teórico implica el producto interno entre la señal recibida y cada base } c_q  \\
&\text{y la simplificación usando propiedades del chirp y del módulo, hasta llegar a una expresión que permite }  \\
&\text{una implementación ediciete mediante una FFT } \\
&\text{1.Producto interno entre la señal recibida }r(nT_s + kT)\text{ y cada una de las formas de onda base} c(nT_s+ kT) \\
&\text{\hspace{0.5cm} Matemáticamente: } \\
&\hspace{1cm} \langle r(nT_s + kT), c(nT_s + kT)|_{s(nT_s) = q} \rangle = \sum_{k=0}^{2^{SF}-1} r(nT_s + kT) \cdot c^*(nT_s + kT)|_{s(nT_s) = q} \quad (11) \\
&\text{\hspace{0.5cm} El valor indica cuánto se parece la señal recibida a cada una de las señales base posible.El } \\
&\text{\hspace{0.5cm} índice q que da el máximo módulo se toma como símbolo detectado  }  \\
&\text{La proyección puede simplificarse utilizando la función  } \varphi(q,k) \text{ tal como se define en las ecuaciones}\\
&\text{ (12) a (15) de [1]}  \\


\end{aligned}
$$

$$
\begin{aligned}
&\textbf{n-Tuple Former} \\[1em]
&\text{El n-Tuple Former es un procedimiento computacional eficiente que implementa la detección óptima} \\
&\text{de símbolos en la modulación Frequency Shift Chirp Modulation (FSCM). Su función es determinar} \\
&\text{cuál de los } 2^{SF} \text{ posibles símbolos fue transmitido, sin necesidad de realizar correlaciones} \\
&\text{directas con cada forma de onda base. Para ello, transforma la señal recibida mediante la} \\
&\text{multiplicación con una función conjugada (down-chirp), seguida de una Transformada Discreta} \\
&\text{de Fourier (DFT). El índice de la máxima magnitud del espectro resultante corresponde al} \\
&\text{símbolo transmitido. Este proceso reemplaza a la correlación exhaustiva y permite recuperar} \\
&\text{directamente la n-tupla binaria original asociada al símbolo, de manera eficiente.}

\end{aligned}

$$
\begin{aligned}
&\textbf{Pasos:} \\[1em]
&\text{1. Multiplicación por down chirp} \\
&\text{\hspace{0.5cm} a. Partiendo de la ecuación (10), se multiplica cada muestra de la señal recibida por un down-chirp conjugado:} \\
&\hspace{1.5cm} d(nT_s + kT) = r(nT_s + kT) \cdot e^{-j2\pi \frac{k^2}{2^{SF}}} \\
&\text{\hspace{0.5cm} elimina la componente cuadrática del chirp} \\[1em]
&\text{2. Aplicación de la DFT (vía FFT) sobre el vector } d(nT_s + kT), \text{ lo que equivale a proyectar sobre cada una} \\
&\text{\hspace{0.5cm} de las formas de onda base:} \\
&\hspace{1.5cm} \langle r, c|_p \rangle = \sum_{k=0}^{2^{SF}-1} d(nT_s + kT) \cdot e^{-j2\pi \frac{pk}{2^{SF}}} \quad (16) \\[1em]

&\text{A partir de las ecuaciones (11) a (16) del paper [1], esta proyección puede expresarse de forma expandida como:} \\[0.5em]
&\hspace{1.5cm} \sum_{k=0}^{2^{SF}-1} 
\underbrace{r(nT_s + kT) \cdot e^{-j2\pi \frac{k^2}{2^{SF}}}}_{\text{down-chirp: } d(nT_s + kT)}
\cdot \frac{1}{\sqrt{2^{SF}}} \cdot e^{-j2\pi \frac{p \cdot k}{2^{SF}}} \\[1em]


&\text{Esta expresión refleja la multiplicación por el down-chirp conjugado y posterior proyección mediante una DFT,} \\
&\text{lo que permite una detección óptima con baja complejidad computacional.}


\end{aligned}
$$



$$
\begin{aligned}
&\text{3. Decisión:} \\
&\text{\hspace{0.5cm} El índice } p \text{ que maximiza el módulo de esta proyección es el símbolo detectado.} \\
&\text{\hspace{0.5cm} Como } p \in \{0, 1, ..., 2^{SF}-1\}, \text{ puede representarse directamente como una tupla de } SF \text{ bits.}\\
\end{aligned}
$$

$$
= \sum_{k=0}^{2^{SF}-1} 
\underbrace{r(nT_s + kT) \cdot e^{-j2\pi \frac{k^2}{2^{SF}}}}_{d(nT_s + kT)}
\cdot \frac{1}{\sqrt{2^{SF}}} e^{-j2\pi p_k \frac{1}{2^{SF}}}
$$

$$
\begin{aligned}
&\textbf{Implementación en Python}
\end{aligned}
$$

In [ ]:
"""
Lógica del algoritmo:

1. Un bloque de SF bits se convierte en un símbolo decimal s ∈ [0, 2^SF - 1].

2. Para cada símbolo s, se genera una forma de onda c[k] = (1/√2^SF) * exp(j2π * ((s + k) mod 2^SF) * k / 2^SF)
   — esta es la chirp modulada, definida en la ecuación (2) del paper.

3. La señal transmitida se reconstruye concatenando todas las chirps.

4. En el receptor (sin canal ruidoso), la señal recibida r[k] es igual a la transmitida.

5. Para detectar cada símbolo:
   a) Se extrae la chirp r_s de longitud 2^SF.
   b) Se multiplica por una chirp conjugada descendente: exp(-j2π k^2 / 2^SF).
   c) Se aplica una FFT al resultado.
   d) El índice donde la magnitud es máxima corresponde al símbolo estimado.

6. Se calcula el SER: proporción de símbolos decodificados incorrectamente.
"""

In [ ]:
import numpy as np

def bits_to_symbols(bit_array, SF):
    #Convierte la lista de bits en un array NumPy y lo reestructura para que cada fila tenga SF bits.
    bit_array = np.array(bit_array).reshape(-1, SF)
    #Crea un array con los pesos binarios, de más significativo a menos (ej: [128, 64, ..., 1] si SF=8).
    powers = 2 ** np.arange(SF)[::-1]
    #Calcula el valor decimal del símbolo multiplicando bits × pesos binarios.
    symbols = bit_array.dot(powers)
    return symbols

In [ ]:
def symbols_to_bits(symbols, SF):
    #Para cada símbolo, aplica una máscara binaria para extraer los bits (de mayor a menor).
    bits = ((symbols[:, None] & (1 << np.arange(SF)[::-1])) > 0).astype(int)
    #Devuelve los bits reordenados en una sola dimensión.
    return bits.reshape(-1)

In [ ]:
def generate_chirp(symbol, SF):
    #Calcula el número total de muestras por chirp.
    N = 2 ** SF
    # Define un vector con índices de tiempo 𝑘=0,1...,𝑁−1
    k = np.arange(N)
    #Calcula la fase de la chirp con desplazamiento según el símbolo
    phase = ((symbol + k) % N) * k
    #Devuelve la chirp normalizada (compleja), de fase cuadrática.
    return np.exp(1j * 2 * np.pi * phase / N) / np.sqrt(N)


In [ ]:

def waveform_former(symbols, SF):
    #Para cada símbolo, genera su chirp y los une todos en una sola señal transmitida.
    return np.concatenate([generate_chirp(s, SF) for s in symbols])

In [ ]:
def n_tuple_former(received_signal, SF):
    #Determina la cantidad de muestras por chirp (N) y cuántos símbolos hay en total.
    N = 2 ** SF
    num_symbols = received_signal.shape[0] // N
    k = np.arange(N)
    down_chirp = np.exp(-1j * 2 * np.pi * k**2 / N)

    detected_symbols = []
    for i in range(num_symbols):
        r_chunk = received_signal[i*N:(i+1)*N]
        mixed = r_chunk * down_chirp
        spectrum = np.fft.fft(mixed)
        detected_symbols.append(np.argmax(np.abs(spectrum)))

    return np.array(detected_symbols)

In [21]:

def simulate_lora_transceiver(SF, total_bits):
    assert total_bits % SF == 0, "El número total de bits debe ser múltiplo de SF"
    N = 2 ** SF

    # Entrada: generación de bits
    tx_bits = np.random.randint(0, 2, total_bits)
    tx_symbols = bits_to_symbols(tx_bits, SF)

    # Waveform Former: generar señal transmitida
    tx_signal = waveform_former(tx_symbols, SF)

    # Canal ideal
    rx_signal = tx_signal.copy()

    # n-Tuple Former: demodulación
    rx_symbols = n_tuple_former(rx_signal, SF)
    rx_bits = symbols_to_bits(rx_symbols, SF)

    # SER y resultado
    symbol_errors = np.sum(tx_symbols != rx_symbols)
    ser = symbol_errors / len(tx_symbols)

    print("Símbolos transmitidos (primeros 16): ", tx_symbols[:16])
    print("Símbolos recibidos (primeros 16):    ", rx_symbols[:16])
    print(f"\nTotal de símbolos: {len(tx_symbols)}")
    print(f"Símbolos erróneos: {symbol_errors}")
    print(f"SER (Symbol Error Rate): {ser:.6f}")

# Ejecutar simulación
SF = 7
num_bits = 1000 * SF
simulate_lora_transceiver(SF, num_bits)

Símbolos transmitidos (primeros 16):  [ 22 113  26  87  34  19 119  49   1  52  41  15  21  28   7 101]
Símbolos recibidos (primeros 16):     [ 22 113  26  87  34  19 119  49   1  52  41  15  21  28   7 101]

Total de símbolos: 1000
Símbolos erróneos: 0
SER (Symbol Error Rate): 0.000000
